<b>Part 1<br>
A description of the problem and a discussion of the background.<br>
Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

Where in London,UK would it be best to open a restaurant. What is the place that has the most reviews, what location seems to be the most popular. Is there a type of restaurant that is more popular than another?<br>

The problem will be solved using the Foursquare API, looking at the usercount and tipcount. This problem will help everyone that wants to open up a restaurant in London and wants to know if it is a viable option where they intend on opening.

<b>Part 2<br>
A description of the data and how it will be used to solve the problem.<br>
Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.<br>

The data that will be used is Foursquare data for London city. It will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [19]:
# Get longitude and latitude for London
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London is 51.5073219, -0.1276474.


In [20]:
# Set up Foursquare
CLIENT_ID = 'WM1QYH4N0ZOMZR35J1YXKNTDYQVKDADBJW5FULQZ3PJ0STZ4' # your Foursquare ID
CLIENT_SECRET = 'JHGZ35XMZU4AZRKDDGN5QF1BFPLEVXOPBDZKQBOTKTGCONAP' # your Foursquare Secret
VERSION = '20191007' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WM1QYH4N0ZOMZR35J1YXKNTDYQVKDADBJW5FULQZ3PJ0STZ4
CLIENT_SECRET:JHGZ35XMZU4AZRKDDGN5QF1BFPLEVXOPBDZKQBOTKTGCONAP


In [21]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')


Restaurant .... OK!


In [22]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WM1QYH4N0ZOMZR35J1YXKNTDYQVKDADBJW5FULQZ3PJ0STZ4&client_secret=JHGZ35XMZU4AZRKDDGN5QF1BFPLEVXOPBDZKQBOTKTGCONAP&ll=51.5073219,-0.1276474&v=20191007&query=Restaurant&radius=50000&limit=100'

In [23]:
results = requests.get(url).json()

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",False,4ac518caf964a520bda520e3,13 Percy St,GB,London,United Kingdom,NaN,1273,"[13 Percy St, London, Greater London, W1T 1DP,...","[{'label': 'display', 'lat': 51.51827036363987...",51.518270,-0.132954,NaN,W1T 1DP,Greater London,The Elysee Restaurant and Roof Garden,v-1570535875,NaN
1,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",False,5001b8e6e4b05e79b0b3b84d,36-44 Brewer St,GB,London,United Kingdom,Lexington St,719,"[36-44 Brewer St (Lexington St), Soho, Greater...","[{'label': 'display', 'lat': 51.51183377093123...",51.511834,-0.135080,NaN,W1F 9TB,Greater London,Bill's Restaurant,v-1570535875,NaN
2,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",False,4d440bf1c3e5f04d0a219620,28 St Martin's Courtyard,GB,Covent Garden,United Kingdom,Upper St Martin's Ln,587,[28 St Martin's Courtyard (Upper St Martin's L...,"[{'label': 'display', 'lat': 51.51255465283461...",51.512555,-0.126600,NaN,WC2E 9AB,Greater London,Bill's Restaurant,v-1570535875,NaN
3,"[{'id': '52e81612bcbc57f1066b7a05', 'name': 'E...",False,4b5344aef964a5200c9527e3,1-5 West St,GB,London,United Kingdom,Litchfield St,610,"[1-5 West St (Litchfield St), London, Greater ...","[{'label': 'display', 'lat': 51.51279973860704...",51.512800,-0.127981,NaN,WC2H 9NQ,Greater London,The Ivy Restaurant,v-1570535875,NaN
4,"[{'id': '58daa1558bbb0b01f18ec1cd', 'name': 'L...",False,4beb16d2183895212eed0acf,31-33 Shepherd Mkt,GB,London,United Kingdom,NaN,1354,"[31-33 Shepherd Mkt, London, Greater London, W...","[{'label': 'display', 'lat': 51.50643435933546...",51.506434,-0.147142,NaN,W1J 7PT,Greater London,Al Hamra Restaurant,v-1570535875,NaN


In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Elysee Restaurant and Roof Garden,Greek Restaurant,13 Percy St,GB,London,United Kingdom,NaN,1273,"[13 Percy St, London, Greater London, W1T 1DP,...","[{'label': 'display', 'lat': 51.51827036363987...",51.518270,-0.132954,NaN,W1T 1DP,Greater London,4ac518caf964a520bda520e3
1,Bill's Restaurant,English Restaurant,36-44 Brewer St,GB,London,United Kingdom,Lexington St,719,"[36-44 Brewer St (Lexington St), Soho, Greater...","[{'label': 'display', 'lat': 51.51183377093123...",51.511834,-0.135080,NaN,W1F 9TB,Greater London,5001b8e6e4b05e79b0b3b84d
2,Bill's Restaurant,English Restaurant,28 St Martin's Courtyard,GB,Covent Garden,United Kingdom,Upper St Martin's Ln,587,[28 St Martin's Courtyard (Upper St Martin's L...,"[{'label': 'display', 'lat': 51.51255465283461...",51.512555,-0.126600,NaN,WC2E 9AB,Greater London,4d440bf1c3e5f04d0a219620
3,The Ivy Restaurant,English Restaurant,1-5 West St,GB,London,United Kingdom,Litchfield St,610,"[1-5 West St (Litchfield St), London, Greater ...","[{'label': 'display', 'lat': 51.51279973860704...",51.512800,-0.127981,NaN,WC2H 9NQ,Greater London,4b5344aef964a5200c9527e3
4,Al Hamra Restaurant,Lebanese Restaurant,31-33 Shepherd Mkt,GB,London,United Kingdom,NaN,1354,"[31-33 Shepherd Mkt, London, Greater London, W...","[{'label': 'display', 'lat': 51.50643435933546...",51.506434,-0.147142,NaN,W1J 7PT,Greater London,4beb16d2183895212eed0acf


In [18]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the The Ritz London

# add a red circle marker to represent the The Ritz London
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='The Ritz London',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map